# 3 - Análise exploratória e Consultas SQL

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import duckdb
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

### Carregando dados

In [ ]:
df = pd.read_parquet('arrests_limpo.parquet', engine='fastparquet')

df = df.replace('(null)', np.nan)

print("--- ESTADO INICIAL (ANTES DO TIDYING) ---")
df.info()
df.head(5)

## Transformações estruturais

In [ ]:
# Não precisa.

## Normalização

In [ ]:
print("Iniciando Padronização (Pré-requisitos)...")

print("Convertendo 'arrest_date' para datetime...")
df['arrest_date'] = pd.to_datetime(df['arrest_date'], errors='coerce')

print("Convertendo coordenadas para numérico...")
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

print("Padronização inicial concluída.")

print("Iniciando Normalização...")

print("Separando 'arrest_date' em componentes...")
df['arrest_year'] = df['arrest_date'].dt.year
df['arrest_month'] = df['arrest_date'].dt.month
df['arrest_day'] = df['arrest_date'].dt.day
df['arrest_day_of_week'] = df['arrest_date'].dt.day_name()
df['arrest_hour'] = df['arrest_date'].dt.hour

print("Removendo coluna aninhada 'geocoded_column'...")
if 'geocoded_column' in df.columns:
    df = df.drop(columns=['geocoded_column'])

print("Normalização concluída.")

## Padronização de tipos

In [ ]:
print("Iniciando Padronização (Otimização)...")

print("Convertendo IDs (arrest_key, arrest_precinct) para string...")
df['arrest_key'] = df['arrest_key'].astype(str)
df['arrest_precinct'] = df['arrest_precinct'].astype(str)

print("Convertendo colunas de alta repetição para 'category'...")
colunas_categoricas = [
    'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc', 'law_code',
    'law_cat_cd', 'arrest_boro', 'jurisdiction_code',
    'age_group', 'perp_sex', 'perp_race', 'arrest_day_of_week'
]

for col in colunas_categoricas:
    if col in df.columns:
        df[col] = df[col].astype('category')

print("Padronização (Parte 2) concluída.")

print("--- ESTADO FINAL (DEPOIS DO TIDYING) ---")
df.info()

## Exportação de dados preparados em Parquet

In [ ]:
print("Exportando dados preparados...")
df.to_parquet('arrests_tidy_prepared.parquet', engine='fastparquet')
print("Arquivo 'arrests_tidy_prepared.parquet' salvo com sucesso!")

print("\nAmostra dos dados finais:")
df.head()